# Fractal VR SDQ-DASS-SSQ-IPQ-VR experience analysis. (in R)


This notebook is running behavioral analysis of Fractal VR data. 


## Socio Demographic Questions check

In [ ]:
############################################################
#  Script:   Recode questionnaire data and print APA tables
#  Author:   <Your Name>
#  Date:     2025‑05‑05
#  title: "Your Report"
#  output: html_document
############################################################



# ------------------ 0. Install & load required packages ------------------
required_pkgs <- c("tidyverse",   # readr, dplyr, stringr, …
                   "janitor",     # quick tabulation / name cleaning
                   "remotes",     # to install GitHub packages
                   "apaTables")   # extra APA helpers
to_install <- setdiff(required_pkgs, rownames(installed.packages()))
if (length(to_install)) install.packages(to_install, dependencies = TRUE)

install.packages("gt")
install.packages("gtsummary")

# papaja lives on GitHub
if (!requireNamespace("papaja", quietly = TRUE)) {
  remotes::install_github("crsh/papaja")
}

suppressPackageStartupMessages({
  library(tidyverse)
  library(janitor)
  library(apaTables)
  library(papaja)   
  library(gt) # For creating clean HTML tables
  library(gtsummary) # For APA-style summary tables
  library(dplyr)  # provides apa_table()

})




Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/yr/7rrnl49x4hlcxhdhq2vmvh7c0000gn/T//RtmplVCxOT/downloaded_packages


installiere auch Abh"angigkeit 'cards'





Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/yr/7rrnl49x4hlcxhdhq2vmvh7c0000gn/T//RtmplVCxOT/downloaded_packages


In [32]:


# ------------------ 1. Read the data ------------------
data_file <- "/Users/jiaonahu/Documents/fractalvr-data/SDQ-all.csv"        # <—‑ change to your CSV path
raw_df <- read_csv(data_file, show_col_types = FALSE) %>% 
  clean_names()                   # lower‑case names with underscores

# ------------------ 2. Recode variables ------------------
df <- raw_df %>%
  mutate(
    subject_id = v_1,
    sex = case_when(
      v_32 == 2 ~ "Female",
      v_32 == 3 ~ "Male",
      TRUE      ~ NA_character_
    ),
    handedness = case_when(
      v_35 == 1 ~ "Right - handed",
      v_35 == 2 ~ "Left - handed",
      TRUE      ~ NA_character_
    ),
    arch_background = recode_factor(
      v_54,
      `1` = "Yes",
      `2` = "No",
      .default = NA_character_
    ),
    envsci_background = recode_factor(
      v_40,
      `1` = "Yes",
      `2` = "No",
      .default = NA_character_
    ),
    income = factor(
      v_42,
      levels = 1:8,
      labels = c("under EUR 1250",
                 "EUR 1250 - 1749",
                 "EUR 1750 - 2249",
                 "EUR 2250 - 2999",
                 "EUR 3000 - 3999",
                 "EUR 4000 - 4999",
                 "over EUR 5000",
                 "no answer")
    ),
    # German‑only native speakers vs all others
    german_native = case_when(
      str_detect(str_to_lower(v_56), "^deu(tsch)?$|^german$") ~ "German only",
      is.na(v_56)                                             ~ NA_character_,
      TRUE                                                    ~ "German + other"
    )
  )

# ------------------ 2b. Exclude specific participants ------------------
excluded_ids <- c("Z2CN1H", "YG0DT2", "JGMJ5P", "MX16R1")
df <- df %>% filter(!subject_id %in% excluded_ids)

# Update subject count
n_subj <- n_distinct(df$subject_id)
message("The data set now contains ", n_subj, " unique participants (after exclusion).")

# ------------------ 3. Integrity check ------------------
n_subj <- n_distinct(df$subject_id)
message("The data set contains ", n_subj,
        " unique participants (expected = 40).")

# ------------------ 4. APA‑formatted frequency tables ------------------
freq_tables <- list(
  Sex                     = df %>% count(sex)               %>% adorn_totals("row"),
  Handedness              = df %>% count(handedness)        %>% adorn_totals("row"),
  Architecture_background = df %>% count(arch_background)   %>% adorn_totals("row"),
  EnvSci_background       = df %>% count(envsci_background) %>% adorn_totals("row"),
  Income                  = df %>% count(income)            %>% adorn_totals("row"),
  Mother_tongue           = df %>% count(german_native)     %>% adorn_totals("row")
)

# quick console preview
knitr::kable(freq_tables[["Sex"]])
knitr::kable(freq_tables[["Handedness"]])
knitr::kable(freq_tables[["Architecture_background"]])
knitr::kable(freq_tables[["EnvSci_background"]])
knitr::kable(freq_tables[["Income"]])
knitr::kable(freq_tables[["Mother_tongue"]])



for (nm in names(freq_tables)) {
  cat("\n\n### ", nm, " ###\n", sep = "")
  print(
    papaja::apa_table(
      freq_tables[[nm]],
      caption = paste0("Frequencies of ", nm, "."),
      format  = if (interactive()) "markdown" else "latex"  # markdown in console, latex when knitting
    )
  )
}

# Convert each frequency table into long format with a 'Variable' column
make_table <- function(tbl, var_name) {
  tbl %>%
    mutate(Variable = var_name) %>%
    select(Variable, everything())
}




The data set now contains 36 unique participants (after exclusion).

The data set contains 36 unique participants (expected = 40).





|sex    |  n|
|:------|--:|
|Female | 15|
|Male   | 21|
|Total  | 36|



|handedness     |  n|
|:--------------|--:|
|Right - handed | 36|
|Total          | 36|



|arch_background |  n|
|:---------------|--:|
|Yes             |  2|
|No              | 34|
|Total           | 36|



|envsci_background |  n|
|:-----------------|--:|
|Yes               |  3|
|No                | 33|
|Total             | 36|



|income          |  n|
|:---------------|--:|
|under EUR 1250  | 15|
|EUR 1250 - 1749 |  7|
|EUR 2250 - 2999 |  5|
|EUR 3000 - 3999 |  4|
|EUR 4000 - 4999 |  2|
|over EUR 5000   |  1|
|no answer       |  2|
|Total           | 36|



|german_native  |  n|
|:--------------|--:|
|German + other |  9|
|German only    | 27|
|Total          | 36|



### Sex ###
[1] "\n\n\n\\begin{table}[tbp]\n\n\\begin{center}\n\\begin{threeparttable}\n\n\\caption{Frequencies of Sex.}\n\n\\begin{tabular}{ll}\n\\toprule\nsex & \\multicolumn{1}{c}{n}\\\\\n\\midrule\nFemale & 15\\\\\nMale & 21\\\\\nTotal & 36\\\\\n\\bottomrule\n\\end{tabular}\n\n\\end{threeparttable}\n\\end{center}\n\n\\end{table}\n\n\n"
attr(,"class")
[1] "knit_asis"
attr(,"knit_cacheable")
[1] NA


### Handedness ###
[1] "\n\n\n\\begin{table}[tbp]\n\n\\begin{center}\n\\begin{threeparttable}\n\n\\caption{Frequencies of Handedness.}\n\n\\begin{tabular}{ll}\n\\toprule\nhandedness & \\multicolumn{1}{c}{n}\\\\\n\\midrule\nRight - handed & 36\\\\\nTotal & 36\\\\\n\\bottomrule\n\\end{tabular}\n\n\\end{threeparttable}\n\\end{center}\n\n\\end{table}\n\n\n"
attr(,"class")
[1] "knit_asis"
attr(,"knit_cacheable")
[1] NA


### Architecture_background ###
[1] "\n\n\n\\begin{table}[tbp]\n\n\\begin{center}\n\\begin{threeparttable}\n\n\\caption{Frequencies of Architecture\\_background.}\n\n\\begi

export APA table

In [33]:
# Create an ungrouped demographic summary table
table_summary <- df %>%
  # Select relevant sociodemographic variables
  select(sex, handedness, arch_background, envsci_background, income, german_native) %>%
  
  # Create summary table showing n (%) for each category
  tbl_summary(
    statistic = list(all_categorical() ~ "{n} ({p}%)"),  # show count and percent
    label = list(  # Rename variables for display
      sex ~ "Sex",
      handedness ~ "Handedness",
      arch_background ~ "Architecture Background",
      envsci_background ~ "Environmental Science Background",
      income ~ "Household Income",
      german_native ~ "German Proficiency"
    ),
    missing = "no"  # Do not display missing values
  ) %>%
  modify_header(label ~ "**Variable**") %>%  # Bold the "Variable" header
  bold_labels()  # Bold all variable labels

# Convert the table into a displayable HTML format using gt
gt_table <- as_gt(table_summary)

# Save the HTML table and open it in the browser (e.g., Safari)
gtsave(gt_table, "sociodemo_summary_nogroup.html")
browseURL("sociodemo_summary_nogroup.html")

## VR experience

In [26]:
install.packages(c("tidyverse", "janitor", "readxl", "gt", "gtsummary"))
if (!requireNamespace("remotes", quietly = TRUE)) install.packages("remotes")
remotes::install_github("crsh/papaja")



Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/yr/7rrnl49x4hlcxhdhq2vmvh7c0000gn/T//RtmplVCxOT/downloaded_packages


Skipping install of 'papaja' from a github remote, the SHA1 (18484f48) has not changed since last install.
  Use `force = TRUE` to force installation



In [40]:
# ------------------ Load Packages ------------------
library(tidyverse)
library(readxl)
library(janitor)
library(gt)
library(gtsummary)

# ------------------ 1. Read the data ------------------
file_path <- "/Users/jiaonahu/Documents/fractalvr-data/experience with VR.xlsx"  
df <- read_excel(file_path) %>% clean_names()

# ------------------ 2. Recode Variables ------------------
df <- df %>%
  mutate(
    subject_id = v_1,  # <<== create subject_id
    vr_experience = recode_factor(
      as.character(v_58),
      `1` = "Ja",
      `2` = "Nein",
      `3` = "Keine Angabe"
    ),
    gaming_frequency = recode_factor(
      as.character(v_70),
      `1` = "Niemals",
      `2` = "Weniger als einmal pro Woche",
      `3` = "Einmal pro Woche",
      `4` = "Mehrmals pro Woche",
      `5` = "Jeden Tag"
    )
  )

# ------------------ 2b. Exclude Specific Participants ------------------
excluded_ids <- c("Z2CN1H", "YG0DT2", "JGMJ5P", "MX16R1")
df <- df %>% filter(!subject_id %in% excluded_ids)

# ------------------ 3. Create Summary Table ------------------
table_summary <- df %>%
  select(vr_experience, gaming_frequency) %>%
  tbl_summary(
    statistic = list(all_categorical() ~ "{n} ({p}%)"),
    label = list(
      vr_experience ~ "VR Experience",
      gaming_frequency ~ "Gaming Frequency"
    ),
    missing = "no"
  ) %>%
  modify_header(label ~ "**Variable**") %>%
  bold_labels()

# ------------------ 4. Export as HTML ------------------
gt_table <- as_gt(table_summary)
gtsave(gt_table, "vr_gaming_summary.html")
browseURL("vr_gaming_summary.html")

In [43]:
# ------------------ Select participants with no VR experience ------------------

# Filter the dataset for participants who answered "Nein" (no) to the VR experience question
# Select only the 'subject_id' column for those participants
no_vr_ids <- df %>%
  filter(vr_experience == "Nein") %>%
  select(subject_id)

# Print the resulting IDs to the console
print(no_vr_ids)

# A tibble: 10 x 1
   subject_id
   <chr>     
 1 AAU5L2    
 2 PVAA50    
 3 MNQJJL    
 4 J3R7L4    
 5 6AAVN4    
 6 MARMAA    
 7 VM3K4U    
 8 MPN87Y    
 9 86L56R    
10 FX4L2F    


In [45]:
# ------------------ User-provided list of subject IDs ------------------
user_ids <- c("0FJ87G", "1LCED7", "5Z6HUD", "6AAVN4", "86L56R", "AAU5L2", "CNYTQ6", "E3R3C1",
              "F3N17K", "FX4L2F", "GE00WM", "GH9FPG", "J3R7L4", "KQT35L", "KWCTKJ", "MARMAA",
              "MNQJJL", "N6FGR8", "RJ70HQ", "RQGUR7", "VVXYT5", "X37HQQ", "XRDKL8", "Y98J0U",
              "Z6WJKG")

# ------------------ Extract IDs from your data where VR experience == "Nein" ------------------
# Convert the column from the previous selection to a simple vector
data_ids <- pull(no_vr_ids, subject_id)

# ------------------ Compare and calculate overlaps ------------------

# Find IDs that appear in both lists
overlap <- intersect(user_ids, data_ids)

# Find IDs that are only in the user-provided list
only_in_user <- setdiff(user_ids, data_ids)

# Find IDs that are only in the data (i.e., not in user list)
only_in_data <- setdiff(data_ids, user_ids)

# ------------------ Print summary of overlap ------------------

# Print basic statistics
cat("✅ Total IDs in user-provided list: ", length(user_ids), "\n")
cat("✅ Total IDs in data with no VR experience: ", length(data_ids), "\n")
cat("🔁 Number of overlapping IDs: ", length(overlap), "\n")
cat("📊 Overlap percentage: ", round(length(overlap) / length(user_ids) * 100, 1), "%\n\n")
cat("📊 Overlap percentage from data perspective: ", round(length(overlap) / length(data_ids) * 100, 1), "%\n")

# Print overlapping IDs
cat("✅ Overlapping IDs:\n")
print(overlap)

# Print IDs only in user list
cat("\n❌ IDs only in user-provided list:\n")
print(only_in_user)

# Print IDs only in the dataset
cat("\n❌ IDs only in data (not in user-provided list):\n")
print(only_in_data)


<U+2705> Total IDs in user-provided list:  25 
<U+2705> Total IDs in data with no VR experience:  10 
<U+0001F501> Number of overlapping IDs:  7 
<U+0001F4CA> Overlap percentage:  28 %

<U+0001F4CA> Overlap percentage from data perspective:  70 %
<U+2705> Overlapping IDs:
[1] "6AAVN4" "86L56R" "AAU5L2" "FX4L2F" "J3R7L4" "MARMAA" "MNQJJL"

<U+274C> IDs only in user-provided list:
 [1] "0FJ87G" "1LCED7" "5Z6HUD" "CNYTQ6" "E3R3C1" "F3N17K" "GE00WM" "GH9FPG"
 [9] "KQT35L" "KWCTKJ" "N6FGR8" "RJ70HQ" "RQGUR7" "VVXYT5" "X37HQQ" "XRDKL8"
[17] "Y98J0U" "Z6WJKG"

<U+274C> IDs only in data (not in user-provided list):
[1] "PVAA50" "VM3K4U" "MPN87Y"


## DASS-21

In [66]:
# --------------------- Setup ---------------------
library(readxl)
library(dplyr)

# 1. Load Excel file
file_path <- "/Users/jiaonahu/Documents/fractalvr-data/DASS-pre.xlsx"
df <- read_excel(file_path)

# 2. Clean ID column
df <- df %>%
  mutate(v_1 = trimws(as.character(v_1)))  # Ensure lfdn is character and trimmed

# 3. Exclude specific participant IDs
excluded_ids <- c("Z2CN1H", "YG0DT2", "JGMJ5P", "MX16R1")
df <- df %>% filter(!v_1 %in% excluded_ids)

# 4. Convert all DASS items (1–4) to DASS standard scale (0–3)
df <- df %>%
  mutate(across(v_72:v_92, ~ . - 1))  # Subtract 1 from each response

# 5. Compute scores
df <- df %>%
  mutate(
    # Depression (Q3, Q5, Q10, Q13, Q16, Q17, Q21)
    Depression_raw = rowSums(select(., v_74, v_76, v_81, v_84, v_87, v_88, v_92), na.rm = TRUE),
    Depression_score = Depression_raw * 2,

    # Anxiety (Q2, Q4, Q7, Q9, Q15, Q19, Q20)
    Anxiety_raw = rowSums(select(., v_73, v_75, v_78, v_80, v_86, v_90, v_91), na.rm = TRUE),
    Anxiety_score = Anxiety_raw * 2,

    # Stress (Q1, Q6, Q8, Q11, Q12, Q14, Q18)
    Stress_raw = rowSums(select(., v_72, v_77, v_79, v_82, v_83, v_85, v_89), na.rm = TRUE),
    Stress_score = Stress_raw * 2
  )

# 6. Classify severity levels
df <- df %>%
  mutate(
    Depression_level = case_when(
      Depression_score <= 9  ~ "Normal",
      Depression_score <= 13 ~ "Mild",
      Depression_score <= 20 ~ "Moderate",
      Depression_score <= 27 ~ "Severe",
      TRUE                   ~ "Extremely Severe"
    ),
    Anxiety_level = case_when(
      Anxiety_score <= 7  ~ "Normal",
      Anxiety_score <= 9  ~ "Mild",
      Anxiety_score <= 14 ~ "Moderate",
      Anxiety_score <= 19 ~ "Severe",
      TRUE                ~ "Extremely Severe"
    ),
    Stress_level = case_when(
      Stress_score <= 14 ~ "Normal",
      Stress_score <= 18 ~ "Mild",
      Stress_score <= 25 ~ "Moderate",
      Stress_score <= 33 ~ "Severe",
      TRUE               ~ "Extremely Severe"
    )
  )

# 7. Print full results
print(df %>% select(v_1,
                    Depression_score, Depression_level,
                    Anxiety_score, Anxiety_level,
                    Stress_score, Stress_level))

# 8. Summary statistics
total_remaining <- nrow(df)
cat("\n✅ Total participants after exclusion:", total_remaining, "\n")

all_normal <- df %>%
  filter(Depression_level == "Normal",
         Anxiety_level == "Normal",
         Stress_level == "Normal") %>%
  nrow()

cat("✅ Participants with all Normal scores:", all_normal, "\n")


# A tibble: 36 x 7
   v_1    Depression_score Depression_level Anxiety_score Anxiety_level
   <chr>             <dbl> <chr>                    <dbl> <chr>        
 1 N6FGR8                0 Normal                       0 Normal       
 2 AAU5L2                0 Normal                       0 Normal       
 3 Y6WJKG                0 Normal                       0 Normal       
 4 1L9FCD               10 Mild                         4 Normal       
 5 KQT35L               12 Mild                         2 Normal       
 6 V4JN59                2 Normal                       0 Normal       
 7 VVXYJ5                0 Normal                       0 Normal       
 8 E3R3C1                2 Normal                       2 Normal       
 9 FA5FCX                2 Normal                       2 Normal       
10 KWCTKJ                0 Normal                       0 Normal       
# i 26 more rows
# i 2 more variables: Stress_score <dbl>, Stress_level <chr>

<U+2705> Total participants after excl

In [67]:
# Identify participants with at least one dimension not "Normal"
not_all_normal <- df %>%
  filter(Depression_level != "Normal" |
         Anxiety_level != "Normal" |
         Stress_level != "Normal")

# Print their IDs
cat("✅ IDs of participants with elevated scores (not all 'Normal'):\n")
print(not_all_normal$v_1)


<U+2705> IDs of participants with elevated scores (not all 'Normal'):
 [1] "1L9FCD" "KQT35L" "PVAA50" "J3R7L4" "F3N17K" "MARMAA" "4VP5UW" "VM3K4U"
 [9] "1LCED7" "86L56R"


In [68]:
# ------------------ 1. Your user-provided ID list ------------------
user_ids <- c("0FJ87G", "1LCED7", "5Z6HUD", "6AAVN4", "86L56R", "AAU5L2", "CNYTQ6", "E3R3C1",
              "F3N17K", "FX4L2F", "GE00WM", "GH9FPG", "J3R7L4", "KQT35L", "KWCTKJ", "MARMAA",
              "MNQJJL", "N6FGR8", "RJ70HQ", "RQGUR7", "VVXYT5", "X37HQQ", "XRDKL8", "Y98J0U",
              "Z6WJKG")

# ------------------ 2. Get IDs of participants with any elevated score ------------------
not_all_normal_ids <- df %>%
  filter(Depression_level != "Normal" |
         Anxiety_level != "Normal" |
         Stress_level != "Normal") %>%
  pull(v_1)

# ------------------ 3. Compare and calculate overlaps ------------------
overlap <- intersect(user_ids, not_all_normal_ids)
only_in_user <- setdiff(user_ids, not_all_normal_ids)
only_in_data <- setdiff(not_all_normal_ids, user_ids)

# ------------------ 4. Print comparison summary ------------------

cat("\n📋 Comparison Summary:\n")
cat("✅ Total IDs in user-provided list: ", length(user_ids), "\n")
cat("✅ Total IDs with elevated DASS scores: ", length(not_all_normal_ids), "\n")
cat("🔁 Number of overlapping IDs: ", length(overlap), "\n")
cat("📊 Overlap percentage (user list): ", round(length(overlap) / length(user_ids) * 100, 1), "%\n")
cat("📊 Overlap percentage (elevated participants): ", round(length(overlap) / length(not_all_normal_ids) * 100, 1), "%\n\n")

cat("✅ Overlapping IDs:\n")
print(overlap)

cat("\n❌ IDs only in user-provided list:\n")
print(only_in_user)

cat("\n❌ IDs only in elevated score group (not in user list):\n")
print(only_in_data)



<U+0001F4CB> Comparison Summary:
<U+2705> Total IDs in user-provided list:  25 
<U+2705> Total IDs with elevated DASS scores:  10 
<U+0001F501> Number of overlapping IDs:  6 
<U+0001F4CA> Overlap percentage (user list):  24 %
<U+0001F4CA> Overlap percentage (elevated participants):  60 %

<U+2705> Overlapping IDs:
[1] "1LCED7" "86L56R" "F3N17K" "J3R7L4" "KQT35L" "MARMAA"

<U+274C> IDs only in user-provided list:
 [1] "0FJ87G" "5Z6HUD" "6AAVN4" "AAU5L2" "CNYTQ6" "E3R3C1" "FX4L2F" "GE00WM"
 [9] "GH9FPG" "KWCTKJ" "MNQJJL" "N6FGR8" "RJ70HQ" "RQGUR7" "VVXYT5" "X37HQQ"
[17] "XRDKL8" "Y98J0U" "Z6WJKG"

<U+274C> IDs only in elevated score group (not in user list):
[1] "1L9FCD" "PVAA50" "4VP5UW" "VM3K4U"


## IPQ_post (PGID 6999382)

In [76]:
# ----------- Setup -----------
library(readxl)
library(dplyr)
library(gt)

# 1. Load Excel file
file_path <- "/Users/jiaonahu/Documents/fractalvr-data/IPQ-Post.xlsx"
df <- read_excel(file_path)

# 2. Reverse-score items (7-point scale): v_20, v_21, v_24, v_26, v_28
reverse_items <- c("v_20", "v_21", "v_24", "v_26", "v_28")
df <- df %>%
  mutate(across(all_of(reverse_items), ~ 8 - .))  # Reverse 1–7 scale

# 3. Exclude specific participant IDs
excluded_ids <- c("Z2CN1H", "YG0DT2", "JGMJ5P", "MX16R1")
df <- df %>% filter(!v_1 %in% excluded_ids)

# 3. Compute subscale scores based on IPQ structure
df <- df %>%
  mutate(
    spatial_presence = rowMeans(select(., v_19, v_20, v_21, v_22, v_23), na.rm = TRUE),
    involvement = rowMeans(select(., v_24, v_25, v_26, v_27), na.rm = TRUE),
    experienced_realism = rowMeans(select(., v_28, v_29, v_30, v_31), na.rm = TRUE),
    general_presence = v_18
  )

# 4. Create summary table (mean of each dimension)
table_summary <- df %>%
  summarise(
    Spatial_Mean = mean(spatial_presence, na.rm = TRUE),
    Involvement_Mean = mean(involvement, na.rm = TRUE),
    Realism_Mean = mean(experienced_realism, na.rm = TRUE),
    Presence_Mean = mean(general_presence, na.rm = TRUE)
  )

# 5. Export as HTML using gt
library(gt)
gt_table <- gt(table_summary)
gtsave(gt_table, "vr_gaming_summary.html")
browseURL("vr_gaming_summary.html")


In [78]:
# 设定阈值：低于 4 被视为觉得不真实
threshold <- 4

# 找出这些参与者
unreal_users <- df %>%
  filter(experienced_realism < threshold) %>%
  select(v_1, experienced_realism)  # v_1 是你的参与者 ID
  
# 3. Exclude specific participant IDs
excluded_ids <- c("Z2CN1H", "YG0DT2", "JGMJ5P", "MX16R1")
df <- df %>% filter(!v_1 %in% excluded_ids)

# 打印结果
cat("🧠 Participants who rated the virtual environment as less realistic (experienced_realism < 4):\n")
print(unreal_users)

<U+0001F9E0> Participants who rated the virtual environment as less realistic (experienced_realism < 4):
# A tibble: 23 x 2
   v_1    experienced_realism
   <chr>                <dbl>
 1 N6FGR8                3.25
 2 KQT35L                1.75
 3 V4JN59                2.5 
 4 VVXYJ5                2.5 
 5 FA5FCX                2.75
 6 KWCTKJ                3   
 7 PVAA50                3.5 
 8 0FJ87G                3.25
 9 MNQJJL                3   
10 Y98JOU                3.25
# i 13 more rows


In [79]:
# ------------------ 1. User-provided ID list ------------------
user_ids <- c("0FJ87G", "1LCED7", "5Z6HUD", "6AAVN4", "86L56R", "AAU5L2", "CNYTQ6", "E3R3C1",
              "F3N17K", "FX4L2F", "GE00WM", "GH9FPG", "J3R7L4", "KQT35L", "KWCTKJ", "MARMAA",
              "MNQJJL", "N6FGR8", "RJ70HQ", "RQGUR7", "VVXYT5", "X37HQQ", "XRDKL8", "Y98J0U",
              "Z6WJKG")

# ------------------ 2. IDs with low experienced realism (< 4) ------------------
low_realism_ids <- c("N6FGR8", "YG0DT2", "KQT35L", "V4JN59", "VVXYJ5", "FA5FCX", 
                     "MX16R1", "KWCTKJ", "PVAA50", "0FJ87G")

# ------------------ 3. Compare lists ------------------
overlap <- intersect(user_ids, low_realism_ids)
only_in_user <- setdiff(user_ids, low_realism_ids)
only_in_low_realism <- setdiff(low_realism_ids, user_ids)

# ------------------ 4. Print summary ------------------
cat("\n🎯 Comparison: User List vs. Low Realism Participants\n")
cat("🔢 Total in user list: ", length(user_ids), "\n")
cat("🔻 Total with low realism: ", length(low_realism_ids), "\n")
cat("🔁 Overlap count: ", length(overlap), "\n")
cat("📊 Overlap (user list): ", round(length(overlap) / length(user_ids) * 100, 1), "%\n")
cat("📊 Overlap (low realism): ", round(length(overlap) / length(low_realism_ids) * 100, 1), "%\n\n")

cat("✅ Overlapping IDs:\n")
print(overlap)

cat("\n🧾 Only in user-provided list:\n")
print(only_in_user)

cat("\n🚨 Only in low realism group (not in user list):\n")
print(only_in_low_realism)



<U+0001F3AF> Comparison: User List vs. Low Realism Participants
<U+0001F522> Total in user list:  25 
<U+0001F53B> Total with low realism:  10 
<U+0001F501> Overlap count:  4 
<U+0001F4CA> Overlap (user list):  16 %
<U+0001F4CA> Overlap (low realism):  40 %

<U+2705> Overlapping IDs:
[1] "0FJ87G" "KQT35L" "KWCTKJ" "N6FGR8"

<U+0001F9FE> Only in user-provided list:
 [1] "1LCED7" "5Z6HUD" "6AAVN4" "86L56R" "AAU5L2" "CNYTQ6" "E3R3C1" "F3N17K"
 [9] "FX4L2F" "GE00WM" "GH9FPG" "J3R7L4" "MARMAA" "MNQJJL" "RJ70HQ" "RQGUR7"
[17] "VVXYT5" "X37HQQ" "XRDKL8" "Y98J0U" "Z6WJKG"

<U+0001F6A8> Only in low realism group (not in user list):
[1] "YG0DT2" "V4JN59" "VVXYJ5" "FA5FCX" "MX16R1" "PVAA50"


In [75]:
# 1. List of participants with low experienced realism
low_realism_ids <- c("N6FGR8", "YG0DT2", "KQT35L", "V4JN59", "VVXYJ5", 
                     "FA5FCX", "MX16R1", "KWCTKJ", "PVAA50", "0FJ87G")

# 2. Your newly provided user list
new_user_ids <- c("1L9FCD", "KQT35L", "PVAA50", "J3R7L4", "F3N17K", 
                  "MARMAA", "4VP5UW", "VM3K4U", "1LCED7", "86L56R")

# 3. Comparison
overlap <- intersect(new_user_ids, low_realism_ids)
only_in_new <- setdiff(new_user_ids, low_realism_ids)
only_in_low_realism <- setdiff(low_realism_ids, new_user_ids)

# 4. Output
cat("\n📋 Comparison Summary:\n")
cat("✅ Total in new user list: ", length(new_user_ids), "\n")
cat("✅ Total with low realism: ", length(low_realism_ids), "\n")
cat("🔁 Overlap count: ", length(overlap), "\n")
cat("📊 Overlap (user list): ", round(length(overlap) / length(new_user_ids) * 100, 1), "%\n")
cat("📊 Overlap (low realism list): ", round(length(overlap) / length(low_realism_ids) * 100, 1), "%\n\n")

cat("✅ Overlapping IDs:\n")
print(overlap)

cat("\n❌ Only in new user list:\n")
print(only_in_new)

cat("\n❌ Only in low realism list (not in new user list):\n")
print(only_in_low_realism)



<U+0001F4CB> Comparison Summary:
<U+2705> Total in new user list:  10 
<U+2705> Total with low realism:  10 
<U+0001F501> Overlap count:  2 
<U+0001F4CA> Overlap (user list):  20 %
<U+0001F4CA> Overlap (low realism list):  20 %

<U+2705> Overlapping IDs:
[1] "KQT35L" "PVAA50"

<U+274C> Only in new user list:
[1] "1L9FCD" "J3R7L4" "F3N17K" "MARMAA" "4VP5UW" "VM3K4U" "1LCED7" "86L56R"

<U+274C> Only in low realism list (not in new user list):
[1] "N6FGR8" "YG0DT2" "V4JN59" "VVXYJ5" "FA5FCX" "MX16R1" "KWCTKJ" "0FJ87G"


In [82]:
# --------------------- Setup ---------------------
library(readxl)
library(dplyr)

# 1. Set file path
file_path <- "/Users/jiaonahu/Documents/fractalvr-data/IPQ-Post.xlsx"

# 2. Read Excel file
df <- read_excel(file_path)

# 3. Ensure ID column is character and trimmed
df <- df %>%
  mutate(v_1 = trimws(as.character(v_1)))

# 4. Define the list of user IDs you want to examine
target_ids <- c("0FJ87G", "1LCED7", "5Z6HUD", "6AAVN4", "86L56R", "AAU5L2", "CNYTQ6", "E3R3C1",
                "F3N17K", "FX4L2F", "GE00WM", "GH9FPG", "J3R7L4", "KQT35L", "KWCTKJ", "MARMAA",
                "MNQJJL", "N6FGR8", "RJ70HQ", "RQGUR7", "VVXYT5", "X37HQQ", "XRDKL8", "Y98J0U",
                "Z6WJKG")

# 5. Filter for selected IDs and calculate average score across v_18 to v_31
average_scores <- df %>%
  filter(v_1 %in% target_ids) %>%
  rowwise() %>%
  mutate(average_score = mean(c_across(v_18:v_31), na.rm = TRUE)) %>%
  ungroup() %>%
  select(v_1, average_score)

# 6. Print results
print(average_scores)


# A tibble: 22 x 2
   v_1    average_score
   <chr>          <dbl>
 1 N6FGR8          3.93
 2 AAU5L2          5.43
 3 KQT35L          3.36
 4 E3R3C1          4.71
 5 KWCTKJ          3.21
 6 RJ70HQ          4.64
 7 0FJ87G          3.14
 8 MNQJJL          3.57
 9 GH9FPG          4.57
10 5Z6HUD          5.07
# i 12 more rows
